In [6]:
import pandas as pd
import sys, io, os
from pathlib import Path


In [34]:
#imput is file name of drug text accounts, returns BERT readable .tsv
def txt_to_tsv(fileName):
        #writes 3 tsv files per input: train, dev, and test

    #dictionaries for labeling data in dataframe
    drug_to_int = {"Mushrooms": 0, "Cannabis": 1, "LSD": 2, "MDMA": 3}
    int_to_drug = {0: "Mushrooms", 1: "Cannabis", 2: "LSD", 3: "MDMA"}
    
    with open('data/%s.txt' % fileName, 'r') as f:
        #clean file of non-sentences and '\n'
        lines = [line.rstrip('\n') for line in f]
        lines = [line for line in lines if len(line) > 45]
        
        #set indices for dividing into train, dev, test
        trainIndex = len(lines) * 8 // 10
        devIndex = (len(lines) // 10) + trainIndex
        
        #generate separate lists of lines
        train = lines[:trainIndex]
        dev = lines[trainIndex:devIndex]
        test = lines[devIndex:]
        
        #preprocess training data for pandas df
        trainRange = range(1, len(train))
        trainData = []
        for num, line in zip(trainRange, train):
            trainData.append([num, drug_to_int[fileName], 'a', line])
            
        #preprocess dev data for pandas df
        devRange = range(1, len(dev))
        devData = []
        for num, line in zip(devRange, dev):
            devData.append([num, drug_to_int[fileName], 'a', line])
            
        #preprocess test data for pandas df
        testRange = range(1, len(test))
        testData = []
        for num, line in zip(testRange, test):
            testData.append([num, line])
            
        #create dataframes
        dfTrain = pd.DataFrame(trainData, columns=['id', 'label', 'filler', 'sentence'])
        dfDev = pd.DataFrame(devData, columns=['id', 'label', 'filler', 'sentence'])
        dfTest = pd.DataFrame(testData, columns=['id', 'sentence'])
        
        #create tsv files
        train_tsv = dfTrain.to_csv(sep='\t', index=False, header=False)
        dev_tsv = dfDev.to_csv(sep='\t', index=False, header=False)
        test_tsv = dfTest.to_csv(sep='\t', index=False, header=True)
        
        #write tsv's
        with open('./data/train.tsv', 'a+') as f:
            f.write(train_tsv)
        
        with open('./data/dev.tsv', 'a+') as f:
            f.write(dev_tsv)
            
        with open('./data/test.tsv', 'a+') as f:
            f.write(test_tsv)
        
                
    

In [35]:
txt_to_tsv('Mushrooms')

In [36]:
txt_to_tsv('LSD')

In [37]:
txt_to_tsv('Cannabis')

In [38]:
txt_to_tsv('MDMA')

In [19]:
def break_txts_into_lines():
    index = 0
    curr_dir = os.getcwd()
    curr_path = Path(curr_dir)
    data_path = curr_path / "data"
    # exclude .DS_Store aka first element
    dirs = os.listdir(data_path)[1:]
    for dirr in dirs:
        subdir_path = data_path / dirr
        # update index to keep track of created files names (so no 2 files created have the same name aka index)
        index = breaker(subdir_path, dirr, index)
        
    
break_txts_into_lines()

created '2681.5' files in '/Users/gkaufmann/Documents/GitHub/NN_final/NN_final/data/Cannabis' location for 'Cannabis' drug
created '5885.5' files in '/Users/gkaufmann/Documents/GitHub/NN_final/NN_final/data/Mushrooms' location for 'Mushrooms' drug
created '2188.0' files in '/Users/gkaufmann/Documents/GitHub/NN_final/NN_final/data/MDMA' location for 'MDMA' drug
created '4189.0' files in '/Users/gkaufmann/Documents/GitHub/NN_final/NN_final/data/LSD' location for 'LSD' drug


In [18]:
def breaker(file_path, drug_name, start_ind):
    index = start_ind + 1
    with open(str(file_path / (drug_name + ".txt")), "r", encoding='utf-8') as file:
        #keep track of whether we add line to file or acc
        count = 1
        acc = ''
        for i, line in enumerate(file):
            #only add if greater than 35 chars in length
            if len(line) > 35: 
                acc += line
                #if count is even we write the two lines to the file
                if not count %2 == 0:
                    with open(str(file_path / (str(start_ind + i//2))) + ".txt", "w+", encoding='utf-8') as newf:
                        newf.write(line)
                    acc = ''
                else:
                    #otherwise we accumulate the two lines in acc
                    acc += line
                index += .5
            count += 1
    print("created '%s' files in '%s' location for '%s' drug" % ((index-start_ind), file_path, drug_name))
    return int(index)
